In [2]:
from PIL import Image
from io import BytesIO
import requests
import boto3
import time
import random
import json
import os
import pandas as pd

In [1]:
import pylab

In [3]:



# bucket_name = "chen-gal-test"
# s3 = boto3.client("s3")
# url = 'https://a0.muscache.com/4ea/air/v2//pictures/e8462648-be49-48f8-a394-70fb9c66f384.jpg?t=r:w2500-h1500-sfit,e:fjpg-c85'
# r = requests.get(url, allow_redirects=True)
# # open('shanghai.jpg', 'wb').write(r.content)

# image = Image.open(BytesIO(r.content))

# image_resized = image.resize((image.size[0]//2,image.size[1]//2))

# filename = 'shanghai'
# image_resized.save('shanghai.jpg')
# # s3.put_object(Body=image_resized, Bucket=bucket_name, Key=filename)

# Load in json file

In [4]:
bucket_name = "chen-gal-test"
s3 = boto3.client("s3")
remote_foler = 'AirbnbData/'

#### Choose the city by changing folder's name

In [5]:
folder = 'Seattle-washington-US'


In [6]:
prefix = remote_foler+folder+'/'+'ws_data'

In [8]:
prefix

'AirbnbData/Seattle-washington-US/ws_data'

In [7]:
def s3_list_files(bucket_name,prefix,layers):
    '''
        Function: find all the file names in a S3 bucket
        Input:
            bucket_name: <string> name of the bucket
            prefix:<string>  the specific file path in the bucket
            layers
        Output:<pd.df> a dataframe includes filenames and folders in the bucket
    '''
    s3_resource = boto3.resource('s3')
    bucket = s3_resource.Bucket(name=bucket_name)
    dict_parse = {}
    i = True
    for obj in bucket.objects.filter(Prefix=prefix):
        string = obj.key.split('/')
        if i:
            dict_parse['full_name'] = []
            dict_parse['full_name'].append(obj.key)            
            for x in range(1,layers):                
                dict_parse[x] = []
                dict_parse[x].append(string[-1*x])
            i = False
            
        for x in range(1,layers):            
            dict_parse['full_name'].append(obj.key)
            dict_parse[x].append(string[-1*x])    

    return pd.DataFrame(dict_parse)


In [8]:
json_df = s3_list_files(bucket_name,prefix,2)

#### load in data from S3 bucket

In [9]:
json_files = json_df.drop([0,1],axis = 0)

In [10]:
json_files

,full_name,1
2,AirbnbData/Seattle-washington-US/ws_data/scrap...,scraped_partial_data_10000
3,AirbnbData/Seattle-washington-US/ws_data/scrap...,scraped_partial_data_12000
4,AirbnbData/Seattle-washington-US/ws_data/scrap...,scraped_partial_data_14000
5,AirbnbData/Seattle-washington-US/ws_data/scrap...,scraped_partial_data_16000
6,AirbnbData/Seattle-washington-US/ws_data/scrap...,scraped_partial_data_18000
7,AirbnbData/Seattle-washington-US/ws_data/scrap...,scraped_partial_data_2000
8,AirbnbData/Seattle-washington-US/ws_data/scrap...,scraped_partial_data_20000
9,AirbnbData/Seattle-washington-US/ws_data/scrap...,scraped_partial_data_22000
10,AirbnbData/Seattle-washington-US/ws_data/scrap...,scraped_partial_data_24000
11,AirbnbData/Seattle-washington-US/ws_data/scrap...,scraped_partial_data_26000


In [11]:
jsons_full = json_files['full_name'].values


In [13]:
jsons_full[7]

'AirbnbData/Seattle-washington-US/ws_data/scraped_partial_data_22000'

In [ ]:
for z in range(0,len(jsons_full)):

    response = s3.get_object(Bucket=bucket_name,
                             Key=jsons_full[z])
    apt_info = json.loads(response['Body'].read())

    i = 0
    num_of_aps = len(apt_info.keys())
    
    with open(logfile_name,'a') as f:
            f.write(f'---------------------Parsing file{jsons_full[z]} ')
    print(f'---------------------Parsing file{jsons_full[z]} ')
    print('Total number of apartments in file{} is {}'.format(jsons_full[z],len(apt_info)))
    
    for j in range(0,num_of_aps):

        k = list(apt_info.keys())[j]
        v = apt_info[k]
        with open(logfile_name,'a') as f:
            f.write(f'---------------------\nApartment{j} | {k}')

        if len(v['info']) == 0:
            with open(logfile_name,'a') as f:
                    f.writelines('No image info for this apt \n')
            continue
        print(f"Processing {j} apartment---------------------------------------------------")    
        image_list = v['info']['photo_urls']
        print(f"Processing the image {k}...")
        print(f" Url: {v['url']}")
        
        for i,img_url in enumerate(image_list):

            try:
                r = requests.get(img_url,allow_redirects=True)
                image = Image.open(BytesIO(r.content))
                image_resized = image.resize((int(image.size[0]//1.2),int(image.size[1]//1.2)))
        #         io = BytesIO()
                filename = 'image/'+folder+ '/'+ k +'_'+str(i)+'.jpg'
                image_resized.save(filename)

                time.sleep(0.2+random.random())
            except:
                with open(logfile_name,'a') as f:
                    f.writelines(f'Cannot open {i} apt images!\n')


        with open(logfile_name,'a') as f:
                    f.writelines(f'Download {i} images for the apt!\n')

    with open(logfile_name,'a') as f:
        f.writelines(f'Downloading Finished!\n')

---------------------Parsing fileAirbnbData/Seattle-washington-US/ws_data/scraped_partial_data_22000 
Total number of apartments in fileAirbnbData/Seattle-washington-US/ws_data/scraped_partial_data_22000 is 2000
Processing 0 apartment---------------------------------------------------
Processing the image 4833521...
 Url: https://www.airbnb.com/rooms/4833521
Processing 3 apartment---------------------------------------------------
Processing the image 16432772...
 Url: https://www.airbnb.com/rooms/16432772
Processing 11 apartment---------------------------------------------------
Processing the image 12629023...
 Url: https://www.airbnb.com/rooms/12629023
Processing 12 apartment---------------------------------------------------
Processing the image 13153219...
 Url: https://www.airbnb.com/rooms/13153219
Processing 13 apartment---------------------------------------------------
Processing the image 6337392...
 Url: https://www.airbnb.com/rooms/6337392
Processing 16 apartment-----------

#### Set log file name 

In [15]:

logfile_name = '../Logs/'+'logfile_image_'+folder+'.txt'
print(f'Logfile name is {logfile_name}')

Logfile name is ../Logs/logfile_image_Seattle-washington-US.txt


#### Create folder for image if it doesn't exist

In [69]:
image_folder = 'image/'+folder+ '/'
if not os.path.exists(image_folder):
    os.makedirs(image_folder)

'image/Seattle-washington-US/'

In [ ]:
i = 0
num_of_aps = len(apt_info.keys())

for j in range(0,num_of_aps):
        
    k = list(apt_info.keys())[j]
    v = apt_info[k]
    with open(logfile_name,'a') as f:
        f.write(f'---------------------\nApartment{j} | {k}')
    
    if len(v['info']) == 0:
        with open(logfile_name,'a') as f:
                f.writelines('No image info for this apt \n')
        continue
    print(f"Processing {j} apartment---------------------------------------------------")    
    image_list = v['info']['photo_urls']
    print(f"Processing the image {k}...")
    print(f" Url: {v['url']}")
    for i,img_url in enumerate(image_list):
        
        try:
            r = requests.get(img_url,allow_redirects=True)

            image = Image.open(BytesIO(r.content))
            image_resized = image.resize((int(image.size[0]//1.5),int(image.size[1]//1.5)))
    #         io = BytesIO()
            filename = 'image/'+folder+ '/'+ k +'_'+str(i)+'.jpg'
            image_resized.save(filename)

            time.sleep(0.2+random.random())
        except:
            with open(logfile_name,'a') as f:
                f.writelines(f'Cannot open {i} apt images!\n')
            
    
    with open(logfile_name,'a') as f:
                f.writelines(f'Download {i} images for the apt!\n')

with open(logfile_name,'a') as f:
    f.writelines(f'Downloading Finished!\n')
        
#         s3.put_object(Body=image_resized, Bucket=bucket_name, Key=filename,ContentType = 'bytearray')
        

Processing 1032 apartment---------------------------------------------------
Processing the image 801680...
 Url: https://www.airbnb.com/rooms/801680
Processing 1033 apartment---------------------------------------------------
Processing the image 6780313...
 Url: https://www.airbnb.com/rooms/6780313
Processing 1035 apartment---------------------------------------------------
Processing the image 16335085...
 Url: https://www.airbnb.com/rooms/16335085
Processing 1037 apartment---------------------------------------------------
Processing the image 16678094...
 Url: https://www.airbnb.com/rooms/16678094
Processing 1038 apartment---------------------------------------------------
Processing the image 17314294...
 Url: https://www.airbnb.com/rooms/17314294
Processing 1040 apartment---------------------------------------------------
Processing the image 19119469...
 Url: https://www.airbnb.com/rooms/19119469
Processing 1041 apartment---------------------------------------------------
Proce

Processing 1095 apartment---------------------------------------------------
Processing the image 276450...
 Url: https://www.airbnb.com/rooms/276450
Processing 1096 apartment---------------------------------------------------
Processing the image 19526556...
 Url: https://www.airbnb.com/rooms/19526556
Processing 1099 apartment---------------------------------------------------
Processing the image 19182289...
 Url: https://www.airbnb.com/rooms/19182289
Processing 1100 apartment---------------------------------------------------
Processing the image 20819710...
 Url: https://www.airbnb.com/rooms/20819710
Processing 1101 apartment---------------------------------------------------
Processing the image 16545081...
 Url: https://www.airbnb.com/rooms/16545081
Processing 1102 apartment---------------------------------------------------
Processing the image 17795007...
 Url: https://www.airbnb.com/rooms/17795007
Processing 1103 apartment---------------------------------------------------
Pro

Processing 1165 apartment---------------------------------------------------
Processing the image 17701434...
 Url: https://www.airbnb.com/rooms/17701434
Processing 1166 apartment---------------------------------------------------
Processing the image 6695...
 Url: https://www.airbnb.com/rooms/6695
Processing 1167 apartment---------------------------------------------------
Processing the image 6576530...
 Url: https://www.airbnb.com/rooms/6576530
Processing 1168 apartment---------------------------------------------------
Processing the image 4593985...
 Url: https://www.airbnb.com/rooms/4593985
Processing 1169 apartment---------------------------------------------------
Processing the image 4593974...
 Url: https://www.airbnb.com/rooms/4593974
Processing 1170 apartment---------------------------------------------------
Processing the image 19716328...
 Url: https://www.airbnb.com/rooms/19716328
Processing 1171 apartment---------------------------------------------------
Processing th

Processing 1222 apartment---------------------------------------------------
Processing the image 20023852...
 Url: https://www.airbnb.com/rooms/20023852
Processing 1223 apartment---------------------------------------------------
Processing the image 1550047...
 Url: https://www.airbnb.com/rooms/1550047
Processing 1224 apartment---------------------------------------------------
Processing the image 15232044...
 Url: https://www.airbnb.com/rooms/15232044
Processing 1225 apartment---------------------------------------------------
Processing the image 17942707...
 Url: https://www.airbnb.com/rooms/17942707
Processing 1226 apartment---------------------------------------------------
Processing the image 7618925...
 Url: https://www.airbnb.com/rooms/7618925
Processing 1227 apartment---------------------------------------------------
Processing the image 8734707...
 Url: https://www.airbnb.com/rooms/8734707
Processing 1228 apartment---------------------------------------------------
Proce

Processing 1289 apartment---------------------------------------------------
Processing the image 13460331...
 Url: https://www.airbnb.com/rooms/13460331
Processing 1290 apartment---------------------------------------------------
Processing the image 5734770...
 Url: https://www.airbnb.com/rooms/5734770
Processing 1291 apartment---------------------------------------------------
Processing the image 639101...
 Url: https://www.airbnb.com/rooms/639101
Processing 1293 apartment---------------------------------------------------
Processing the image 6168687...
 Url: https://www.airbnb.com/rooms/6168687
Processing 1294 apartment---------------------------------------------------
Processing the image 12979965...
 Url: https://www.airbnb.com/rooms/12979965
Processing 1295 apartment---------------------------------------------------
Processing the image 8129715...
 Url: https://www.airbnb.com/rooms/8129715
Processing 1297 apartment---------------------------------------------------
Processin